**Contents**

01 Importing Libraries and Dataset

02 Orders data check

03 Products data check

04 Data Merge

04.1 Checking Key Variable

04.2 Merging Dataframes

05 Export

# 01. Importing Libraries and Dataset

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Set Path
path = r'C:\Users\Forrest\Desktop\Work\CareerFoundry\Python\2022-10 Instacart Basket Analysis'

In [5]:
# Import orders_products_combined
df_ords = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

# 02. Orders data check

In [11]:
df_ords.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both
5,2398795,1,2,3,7,15.0,196,1,1,both
6,2398795,1,2,3,7,15.0,10258,2,0,both
7,2398795,1,2,3,7,15.0,12427,3,1,both
8,2398795,1,2,3,7,15.0,13176,4,0,both
9,2398795,1,2,3,7,15.0,26088,5,1,both


In [6]:
df_ords.shape

(32434489, 10)

# 03. Products data check

In [37]:
# Import products_cleaned
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_cleaned.csv'), index_col = False)

In [15]:
df_prods.shape

(49672, 5)

In [38]:
df_prods.head(10)

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3
5,6,Dry Nose Oil,11,11,2.6
6,7,Pure Coconut Water With Orange,98,7,4.4
7,8,Cut Russet Potatoes Steam N' Mash,116,1,1.1
8,9,Light Strawberry Blueberry Yogurt,120,16,7.0
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7,8.4


# 04. Data merge

In [96]:
# Dropping _merge column on df_ords
df_ords.drop('_merge', axis=1, inplace=True)

Product_id is the variable that df_ords and df_prods have in common. This will be key variable for merge.

## 04.1 Checking key variable

In [62]:
#Checking uniqueness of product_id
df_prods[df_prods.duplicated(['product_id'])]

,product_id,product_name,aisle_id,department_id,prices
6785,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26505,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


Two product_id duplicates found. This may be a problem if we merge on product_id

In [58]:
#Checking first duplicate
df_prods[df_prods['product_id'] == 6800]

,product_id,product_name,aisle_id,department_id,prices
6784,6800,Revive Zero Vitamin Water,64,7,6.4
6785,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0


In [79]:
#Checking second duplicate
df_prods[df_prods['product_id'] == 26520]

,product_id,product_name,aisle_id,department_id,prices
26504,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26505,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [86]:
#Checking contiguous product_ids
df_prods[df_prods['product_id'] == 6799]

,product_id,product_name,aisle_id,department_id,prices


In [87]:
df_prods[df_prods['product_id'] == 26519]

,product_id,product_name,aisle_id,department_id,prices


There are no products with product_id 6799 or 26519. Duplicates can be reassigned.

In [90]:
#Checking existence of product_ids 6799, 6800, 26519, and 26520 in df_ords
df_ords[df_ords['product_id'] == 6799].shape

(1978, 10)

In [91]:
df_ords[df_ords['product_id'] == 6800].shape

(8, 10)

In [92]:
df_ords[df_ords['product_id'] == 26519].shape

(51, 10)

In [93]:
df_ords[df_ords['product_id'] == 26520].shape

(562, 10)

All 4 product_id are present in df_ords dataframe. 

**A left join is needed to preserve the product_ids in df_ords that are not present in df_prods.** A full join is unnecessary since we don't need records of products that have never been ordered.

**More investigation is needed to figure out which product_ids to reassign**

## 04.2 Merging dataframes

In [99]:
# Merge test
df_ords.merge(df_prods, how = 'left', on = 'product_id', indicator = True)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77.0,7.0,9.0,both
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,23.0,19.0,4.4,both
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7,both
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32435054,2977660,206209,13,1,12,7.0,14197,5,1,Tomato Paste,9.0,9.0,5.6,both
32435055,2977660,206209,13,1,12,7.0,38730,6,0,Brownie Crunch High Protein Bar,3.0,19.0,5.9,both
32435056,2977660,206209,13,1,12,7.0,31477,7,0,High Protein Bar Chunky Peanut Butter,3.0,19.0,4.2,both
32435057,2977660,206209,13,1,12,7.0,6567,8,0,Chocolate Peanut Butter Protein Bar,3.0,19.0,4.9,both


In [100]:
# Final merge
df_merged = df_ords.merge(df_prods, how = 'left', on = 'product_id', indicator = True)

In [103]:
# Checking merge results
df_merged['_merge'].value_counts()

both          32404859
left_only        30200
right_only           0
Name: _merge, dtype: int64

In [105]:
# Checking merged df shape
df_merged.shape

(32435059, 14)

# 05. Export

In [106]:
# Exporting merged df as pickle
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_master.pkl'))